In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
df = pd.read_csv('neelabh_data_with_clusters.csv')
df['datetime'] = pd.to_datetime(df['datetime'])
df['hour'] = df['datetime'].dt.hour
df['weekday'] = df['datetime'].dt.dayofweek
df['month'] = df['datetime'].dt.month
df['date'] = df['datetime'].dt.day

In [3]:
df.head()

,datetime,LATITUDE,LONGITUDE,clusterID,hour,weekday,month,date
0,2017-04-27 18:01:54,32.733402,-97.113254,8,18,3,4,27
1,2017-04-27 17:58:30,32.733626,-97.109255,8,17,3,4,27
2,2017-04-27 09:54:03,32.733423,-97.103172,1,9,3,4,27
3,2017-04-27 09:49:10,32.733399,-97.103327,1,9,3,4,27
4,2017-04-26 22:55:31,32.733014,-97.103329,1,22,2,4,26


In [4]:
weather_df = pd.read_excel('Arlington_Weather.xlsx')
weather_df['day'] = weather_df['DATE'].dt.dayofweek
weather_df['month'] = weather_df['DATE'].dt.month
weather_df['date'] = weather_df['DATE'].dt.day

In [5]:
weather_df.head()

,DATE,PRCP,TAVG,TMAX,TMIN,TOBS,day,month,date
0,2017-01-01,0.00,58.0,72,44,44,6,1,1
1,2017-01-02,0.45,53.0,62,44,54,0,1,2
2,2017-01-03,0.00,60.0,71,49,50,1,1,3
3,2017-01-04,0.00,42.5,53,32,32,2,1,4
4,2017-01-05,0.00,34.0,37,31,37,3,1,5


In [6]:
prcp = []
tavg = []
for i in range(len(df)):
    prcp.append(weather_df[((weather_df['month']==df.loc[i]['month']) & (weather_df['date']==df.loc[i]['date']))]['PRCP'].values[0])
    tavg.append(weather_df[((weather_df['month']==df.loc[i]['month']) & (weather_df['date']==df.loc[i]['date']))]['TAVG'].values[0])
PRCP = pd.DataFrame(np.array(prcp), columns=['PRCP'])
TAVG = pd.DataFrame(np.array(tavg), columns=['TAVG'])
df = pd.concat([df, PRCP, TAVG], axis=1)

In [ ]:
df['clusterID'].unique()

In [ ]:
sns.pairplot(df[['clusterID', 'hour', 'weekday', 'PRCP', 'TAVG']], hue='clusterID', palette=["#9b59b6", "#3498db", "#95a5a6", "#e74c3c", "#34495e", "#2ecc71", "darkgrey"])

In [ ]:
flatui = ["#9b59b6", "#3498db", "#95a5a6", "#e74c3c", "#34495e", "#2ecc71", "#ffff00"]
sns.palplot(sns.color_palette(flatui))

In [ ]:
sns.heatmap(df.corr(), annot = True, linewidth=.9)

In [7]:
df.head()

,datetime,LATITUDE,LONGITUDE,clusterID,hour,weekday,month,date,PRCP,TAVG
0,2017-04-27 18:01:54,32.733402,-97.113254,8,18,3,4,27,0.03,59.5
1,2017-04-27 17:58:30,32.733626,-97.109255,8,17,3,4,27,0.03,59.5
2,2017-04-27 09:54:03,32.733423,-97.103172,1,9,3,4,27,0.03,59.5
3,2017-04-27 09:49:10,32.733399,-97.103327,1,9,3,4,27,0.03,59.5
4,2017-04-26 22:55:31,32.733014,-97.103329,1,22,2,4,26,0.40,72.0


In [17]:
# hour = to_categorical(df['hour'].as_matrix())
# weekday = to_categorical(df['weekday'].as_matrix())
month = to_categorical(df['month'].as_matrix())
date = to_categorical(df['date'].as_matrix())
# latitude = (df['LATITUDE']/df['LATITUDE'].max()).as_matrix().reshape(-1,1)
# longitude = (df['LONGITUDE']/df['LONGITUDE'].max()).as_matrix().reshape(-1,1)
latitude = df['LATITUDE'].as_matrix().reshape(-1,1)
longitude = df['LONGITUDE'].as_matrix().reshape(-1,1)
prcp = df['PRCP'].as_matrix().reshape(-1,1)
tavg = (df['TAVG']/df['TAVG'].max()).as_matrix().reshape(-1,1)
hour = (df['hour']/df['hour'].max()).as_matrix().reshape(-1,1)
weekday = (df['weekday']/df['weekday'].max()).as_matrix().reshape(-1,1)

In [18]:
X = np.concatenate((latitude, longitude, hour, weekday, prcp, tavg), axis=1)
y = df['clusterID'].as_matrix()
y_cat = to_categorical(y)
y_cat = np.delete(y_cat, 1, 1)

In [19]:
X[0,:]

array([  3.27334024e+01,  -9.71132539e+01,   7.82608696e-01,
         5.00000000e-01,   3.00000000e-02,   7.93333333e-01])

In [20]:
X.shape

(4466, 6)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.3)

In [22]:
y_train.shape

(3126, 10)

In [23]:
df[df['clusterID']==8].shape

(2688, 10)

In [24]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, SGD
from keras import backend as K

In [27]:
K.clear_session()
model = Sequential()
model.add(Dense(10, input_shape=(X_train.shape[1], ), activation='tanh'))
model.add(Dense(10, activation='tanh'))
model.add(Dense(10, activation='tanh'))
model.add(Dense(10, activation='softmax'))
model.compile(Adam(lr=0.03), loss=
             'categorical_crossentropy', metrics=['accuracy'])

In [28]:
model.fit(X_train, y_train, epochs=20, validation_split=0.2, batch_size=16)

Train on 2500 samples, validate on 626 samples
Epoch 1/20
2500/2500 [==============================] - 0s - loss: 0.5133 - acc: 0.6048 - val_loss: 0.4965 - val_acc: 0.5911

In [ ]:
results = model.evaluate(X_test, y_test)

In [ ]:
print(results)